# Hubble Flow Velocity Correction
* Develop the function to correct the local velocity field.
* It is necessary to consider this correction to calculate the luminosity distance of the galaxies with $z \lesssim 0.02$.
* This code follow the formalism of Mould et al. 2000, ApJ, 529, 786.
* The code is also developed with the help from [mould_distance.pro](https://github.com/moustakas/impro/blob/master/pro/galaxy/mould_distance.pro) by [moustakas](https://github.com/moustakas).

In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM

ls_km   = 2.99792458e5 # km/s
deg2rad = np.pi / 180.

In [2]:
from sgAstro import HubbleFlowDistance, z2DL

"""
ra_hms  = "00h54m03.6s"
dec_dms = "+73d05m12s"
c_obj   = SkyCoord(ra_hms, dec_dms)
ra   = c_obj.ra.deg
dec  = c_obj.dec.deg
"""

ra, dec = 224.421583333, -43.1321111111
v_h  = 4875 #ls_km * z
z    = v_h / ls_km
print HubbleFlowDistance(z, ra, dec, verbose=True)

z2DL(5610.6/ls_km)

The object is on Great Attractor
Brute force calculation...
  v_infall(Virgo): 4812.2
  v_infall(Virgo + Great Attractor): 5636.3
  v_infall(Virgo + Great Attractor + Shapley Supercluster): 5610.6
**The object is on No.2 attractor.
**We ignore the assumption that the object should follow the stream, made by Mould et al. (2000).
**But this velocity should be close to the NED value.
(83.93460468643624, 2)
The redshift is too small. The correction for peculiar velocity is necessary!
The function HubbleFlowDistance() can be used.


83.93510709407042

In [43]:
def z2DL(z, H0=67.8, Om0=0.308, verbose=True):
    '''
    This function calculate the luminosity distance from the redshift.
    The default cosmology comes from Planck Collaboration (2015).

    Parameters
    ----------
    z : float
        The redshift
    H0 : float
        The Hubble constant, default: 67.8
    Om0 : float
        Omega_0, default: 0.308

    Returns
    -------
    DL : float
        The luminosity distance, unit: Mpc.

    Notes
    -----
    None.
    '''
    if (z<0.02) & verbose:
        print("The redshift is too small. The correction for peculiar velocity is necessary!")
        print("The function HubbleFlowDistance() can be used.")
    cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)
    DL = cosmo.luminosity_distance(z).value #Luminosity distance in unit Mpc.
    return DL

def ApexConversionCoefficient(v_a, l_a, b_a):
    """
    The conversion coefficient of heliocentric radial velocities to 
    a solar apex of direction (l_a, b_a) and amplitude v_a in any
    reference frame can be expressed as:
        v_corr = v_h + v_a * [cos(b) cos(b_a) cos(l - l_a) + sin(b) sin(b_a)],
    which can be alwayse converted to:
        v_corr = v_h + X * cos(l) cos(b) + Y * sin(l) cos(b) + Z * sin(b).
    Therefore, the coefficients are:
        X = v_a * cos(l_a) * cos(b_a)
        Y = v_a * sin(l_a) * cos(b_a)
        Z = v_a * sin(b_a)
    
    Parameters
    ----------
    v_a : float
        The the velocity amplitude of the apex, units: km/s.
    l_a, b_a : float, float
        The coordinate (longitude, latitude) of the apex, units: degree.
        
    Returns
    -------
    [X, Y, Z] : list
        The three coefficients.
        
    Notes
    -----
    The details are clearly presented in Courteau & van den Bergh, ApJ, 118, 337 (1999).
    """
    l_a = l_a * deg2rad
    b_a = b_a * deg2rad
    X = v_a * np.cos(l_a) * np.cos(b_a)
    Y = v_a * np.sin(l_a) * np.cos(b_a)
    Z = v_a * np.sin(b_a)
    return [X, Y, Z]

def v_LocalGroup(v_h, l, b, v_a=316., l_a=93., b_a=-4., verbose=False):
    """
    Correct the observed heliocentric velocity to the centroid of the Local Group.
    
    Parameters
    ----------
    v_h : float
        The heliocentric velocity of the target, units: km/s.
    l, b : float, float
        The coordinate (longitude, latitude) of the target, units: degree.
    v_a : float; default: 316.
        The apex velocity of the Local Group, units: km/s.
    l_a, b_a : float, float; default: (93, -4)
        The coordinate (longitude, latitude) of the Local Group apex, units: degree.
        
    Returns
    -------
    v_lg : float
        The radial velocity with respect to the centroid of the Local Group, units: km/s.
        
    Notes
    -----
    The default values of Local Group apex is adopted from Karachentsev & Makarov, AJ, 
    111, 794 (1996) in order to be consistent with the NASA/IPAC Extragalactic Database.
    """
    X, Y, Z = ApexConversionCoefficient(v_a, l_a, b_a)
    if verbose:
        print "X={0:.4f}, Y={1:.4f}, Z={2:.4f}".format(X, Y, Z)
    l = l * deg2rad
    b = b * deg2rad
    v_lg = v_h + X * np.cos(l) * np.cos(b) + Y * np.sin(l) * np.cos(b) + \
           Z * np.sin(b)
    return v_lg

def ang_distance(ra1, dec1, ra2, dec2):
    """
    Angular distance.

    Parameters
    ----------
    ra1, dec1 : float, float
        The coordinates of the first object, units: degree.
    ra2, dec2 : float, float
        The coordinates of the second object, units: degree.

    Returns
    -------
    theta : degree
        The angular distance, units: radian.

    Notes
    -----
    None.
    """
    c1  = SkyCoord(ra1, dec1, frame='icrs', unit='deg')
    c2  = SkyCoord(ra2, dec2, frame='icrs', unit='deg')
    sep = c1.separation(c2)
    theta = sep.deg * deg2rad
    return theta

def vel_distance(v1, v2, theta):
    """
    Velocity distance.  The Equation (2) of Mould et al. (2000).
    
    Parameters
    ----------
    v1 : float
        The velocity of the first object, units: km/s.
    v2 : float
        The velocity of the second object, units: km/s.
    theta : float
        The angular distance of the two objects, units: radian.
    
    Returns
    -------
    roa : float
        The velocity distance, units: km/s.
    
    Notes
    -----
    None.
    """
    roa = np.sqrt(v1**2 + v2**2 - 2. * v1 * v2 * np.cos(theta))
    return roa

class Attractor(object):
    """
    The attractor introduces the inflow affecting the targets.  
    The effect of multiple attractors can be linearly added.
    
    Parameters
    ----------
    ra, dec : float, float
        The ra and dec of the attractor, units: degree.
    v_helio : float
        The observed mean heliocentric velocity of the attractor, units: km/s.
    v_LG : float
        The Velocity corrected to the centroid of the Local Group, units: km/s.
    v_fid : float
        Adopted model infall velocity at the position of the LG, units: km/s.
    radius : float
        Assumed cluster radius, units: degree.
    v_range: [float, float]
        Velocity range collapsed for the cluster core (heliocentric), units: km/s. 
        The radius and range give the partial cone that is zeroed to the attractor 
        center in the flow-field program.
    """
    def __init__(self, ra, dec, v_helio, v_LG, v_fid, radius, v_range):
        self.ra = ra
        self.dec = dec
        self.v_helio = v_helio
        self.v_LG    = v_LG
        self.v_fid   = v_fid
        self.radius  = radius
        self.v_range = v_range
    
    def flag_onstream(self, v_LG, ra, dec):
        """
        Flag whether the object is on the stream of the attractor.
        
        Parameters
        ----------
        v_LG : float
            The velocity of the object with respect to the local group, units: km/s.
        ra, dec : float, float
            The coordinates of the object, units: degree.
        
        Returns
        -------
        flag : bool
            Flag whether the object is on the stream (True) or not (False).
            
        Notes
        -----
        None.
        """
        theta = ang_distance(self.ra, self.dec, ra, dec)
        r0a   = vel_distance(self.v_LG, v_LG, theta)
        if (theta < self.radius) and ((self.v_LG - r0a) > self.v_range[0]) \
           and ((self.v_LG + r0a) < self.v_range[1]):
            flag = True
        else:
            flag = False
        return flag
    
    def v_Infall(self, v_LG, ra, dec, gamma=2):
        """
        The infall velocity of the object due to this attractor.
        
        Parameters
        ----------
        v_LG : float
            The velocity of the object with respect to the local group, units: km/s.
        ra, dec : float, float
            The coordinates of the object, units: degree.
        gamma : float
            The slope of the attractor's density profile.
        
        Returns
        -------
        v_inf : bool
            The infall velocity, units: km/s.
            
        Notes
        -----
        None.
        """
        theta = ang_distance(self.ra, self.dec, ra, dec)
        r0a   = vel_distance(self.v_LG, v_LG, theta)
        if self.flag_onstream(v_LG, ra, dec):
            v_inf = 0
        else:
            v_inf = self.v_fid * np.cos(theta) + self.v_fid * \
                    (v_LG - self.v_LG * np.cos(theta)) / r0a * \
                    (r0a / self.v_LG)**(1 - gamma)
        return v_inf
    
def v_Cosmic(v_h, ra, dec, verbose=False):
    """
    Correct the pecular velocity due to the local structures assuming 
    the 3-attractor flow model of Mould et al., ApJ, 529, 786 (2000).
    
    The conversion to the radial velocity with respect to the centroid
    of the Local Group is based on Karachentsev and Makarov, AJ, 111, 
    794 (1996).  
    
    The conversion method adopted here is exactly the same as what used 
    in NASA/IPAC Extragalactic Database.
    
    Parameters
    ----------
    v_h : float
        float
        The heliocentric velocity of the target, units: km/s.
    ra, dec : float, float
        The coordinate (R.A., Dec.) of the target, units: degree.
    verbose : bool; default: False
        Print the details of the results if True.
        
    Returns
    -------
    v_cosmic : float
        The cosmic velocity of the target, which can be used to calculate 
        the luminosity distance, units: km/s.
    flag_v : int
        The flag of the cosmic velocity:
           -1 -- Normal result.
            0 -- The object is on more than one stream, which is probably not possible...
            1 -- The object is on Virgo stream.
            2 -- The object is on Great Attractor stream.
            3 -- The object is on Shapley Supercluster.
        For the cases except 0, the velocity cannot be used to calculate
        the luminosity distance.
    
    Notes
    -----
    The v_cosmic is consistent with NED results of V (Virgo + GA + Shapley).
    """
    #-> Virgo 
    c_vir   = SkyCoord('12:28:19', '+12:40:00', unit=(u.hourangle, u.deg))
    ra_vir  = c_vir.ra.deg
    dec_vir = c_vir.dec.deg
    virgo   = Attractor(ra_vir, dec_vir, 1035, 957, 200, 10, [600, 2300])

    #-> Great Attractor
    c_ga   = SkyCoord('13:20:00', '-44:00:00', unit=(u.hourangle, u.deg))
    ra_ga  = c_ga.ra.deg
    dec_ga = c_ga.dec.deg
    ga     = Attractor(ra_ga, dec_ga, 4600, 4380, 400, 10, [2600, 6600])

    #-> Shapley Supercluster
    c_shap   = SkyCoord('13:30:00', '-31:00:00', unit=(u.hourangle, u.deg))
    ra_shap  = c_shap.ra.deg
    dec_shap = c_shap.dec.deg
    shap     = Attractor(ra_shap, dec_shap, 13800, 13600, 85, 12, [10000, 16000])

    c_obj = SkyCoord(ra, dec, unit="deg")
    l     = c_obj.galactic.l.deg
    b     = c_obj.galactic.b.deg
    v_LG  = v_LocalGroup(v_h, l, b)
    nameList = ["Virgo", "Great Attractor", "Shapley Supercluster"]
    flagList = [virgo.flag_onstream(v_LG, ra, dec), 
                ga.flag_onstream(v_LG, ra, dec),
                shap.flag_onstream(v_LG, ra, dec)]
    if np.sum(flagList) == 0:
        infallList = [virgo.v_Infall(v_LG, ra, dec),
                      ga.v_Infall(v_LG, ra, dec),
                      shap.v_Infall(v_LG, ra, dec)]
        v_cosmic = v_LG + np.sum(infallList)
        flag_v = -1
        if verbose:
            print("Normal calculation...")
            vDetailList = [v_LG + infallList[0],
                           v_LG + infallList[0] + infallList[1],
                           v_cosmic]
            for loop in range(len(infallList)):
                print("v_infall({0}): {1:.1f}".format(nameList[loop], vDetailList[loop]))
    elif np.sum(flagList) != 1:
        v_cosmic = -1
        flag_v   = 0
        if verbose:
            print("The object is on more than one stream...")
    else:
        vlgList  = [virgo.v_LG, ga.v_LG, shap.v_LG]
        idx = flagList.index(1)
        v_cosmic = vlgList[idx]
        flag_v = idx + 1
        if verbose:
            print("The object is on {0}".format(nameList[idx]))
    return v_cosmic, flag_v

def HubbleFlowDistance(z, ra, dec, H0=67.8, Om0=0.308, verbose=False):
    """
    The Hubble flow distance calculated with the cosmic velocity 
    of the target after correcting the peculiar velocity assuming 
    a 3-attractor flow model of Mould et al., ApJ, 529, 786 (2000).
    
    The conversion to the radial velocity with respect to the centroid
    of the Local Group is based on Karachentsev and Makarov, AJ, 111, 
    794 (1996).  
    
    The conversion method adopted here is exactly the same as what used 
    in NASA/IPAC Extragalactic Database, although we use the Planck2015 
    cosmology by default.
    
    Parameters
    ----------
    z : float
        Redshift.
    ra, dec : float, float
        The coordinate (R.A., Dec.) of the target, units: degree.
    H0 : float; default: 67.8
        The Hubble constant.
    Om0 : float; default: 0.308
        The dimensionless density of the baryonic matter, Omega_0. 
    verbose : bool; default: False
        Print the details of the results if True.
        
    Returns
    -------
    d_hf : float
        The 
    
    Notes
    -----
    None.
    """
    v_h = ls_km * z # Convert to heliocentric velocity
    v_cosmic, flag_v = v_Cosmic(v_h, ra, dec, verbose)
    if flag_v == -1:
        z_cosmic = v_cosmic / ls_km
        d_hf  = z2DL(z_cosmic, H0=H0, Om0=Om0, verbose=False) #Luminosity distance in unit Mpc.
    else:
        d_hf = np.nan
        if verbose:
            print("The object is on the stream(s) of the attractor ({0}).".format(flag_v))
    return d_hf
    

ra_hms  = '12h28m19s' #"00h54m03.6s"
dec_dms = '+12d40m00s' #"+73d05m12s"
c_obj   = SkyCoord(ra_hms, dec_dms)
ra   = c_obj.ra.deg
dec  = c_obj.dec.deg
z    = 1035 / ls_km #0.015698
v_h  = 1035 #ls_km * z
v_c  = v_Cosmic(v_h, ra, dec)[0]
z_c  = v_c / ls_km
print HubbleFlowDistance(z, ra, dec, verbose=True)
print z_c

The object is on Virgo
The object is on the stream(s) of the attractor (1).
nan
0.00319220839105


In [63]:
#-> Virgo 
c_vir   = SkyCoord('12:28:19', '+12:40:00', unit=(u.hourangle, u.deg))
ra_vir  = c_vir.ra.deg
dec_vir = c_vir.dec.deg
virgo   = Attractor(ra_vir, dec_vir, 1035, 957, 200, 10, [600, 2300])

#-> Great Attractor
c_ga   = SkyCoord('13:20:00', '-44:00:00', unit=(u.hourangle, u.deg))
ra_ga  = c_ga.ra.deg
dec_ga = c_ga.dec.deg
ga     = Attractor(ra_ga, dec_ga, 4600, 4380, 400, 10, [2600, 6600])

#-> Shapley Supercluster
c_shap   = SkyCoord('13:30:00', '-31:00:00', unit=(u.hourangle, u.deg))
ra_shap  = c_shap.ra.deg
dec_shap = c_shap.dec.deg
shap     = Attractor(ra_shap, dec_shap, 13800, 13600, 85, 12, [10000, 16000])


ra_hms  = "04h13m49.7s"
dec_dms = "-32d00m25s"
c_obj   = SkyCoord(ra_hms, dec_dms)
ra   = c_obj.ra.deg
dec  = c_obj.dec.deg
l    = c_obj.galactic.l.deg
b    = c_obj.galactic.b.deg
print l, b
z    = 0.011908
v_h  = 3570 #ls_km * z
v_LG = v_LocalGroup(v_h, l, b)
print "v_h: {0:.4f}".format(v_h)
print "v_LG: {0:.4f}".format(v_LG)

flagList = [virgo.flag_onstream(v_LG, ra, dec), 
            ga.flag_onstream(v_LG, ra, dec),
            shap.flag_onstream(v_LG, ra, dec)]
infallList = [virgo.v_Infall(v_LG, ra, dec),
              ga.v_Infall(v_LG, ra, dec),
              shap.v_Infall(v_LG, ra, dec)]
print "Virgo Infall only: {0:.4f}".format(v_LG + infallList[0])
print "Virgo + GA only: {0:.4f}".format(v_LG + infallList[0] + infallList[1])
vlgList = [virgo.v_LG, ga.v_LG, shap.v_LG]
if np.sum(flagList) == 0:
    print "sum"
    v_cosmic = v_LG + np.sum(infallList)
elif np.sum(flagList) != 1:
    raise RuntimeError("The object is on more than one stream...")
else:
    idx = flagList.index(1)
    v_cosmic = vlgList[idx]

print "Virgo + GA + Shapley: {0:.4f}".format(v_cosmic)

4.04886671636 -0.804860750659
v_h: 3570.0000
v_LG: 3468.0203
virgo: -68.9722
GA: 168.7516
Shapley: 13.9737
sum
v_cosmic: 3581.7734


In [72]:
print v_h + 316 * np.sin(l) * np.cos(b)
print v_LocalGroup(v_h, l, b)
print v_LocalGroup(v_h, l, b, v_a=300., l_a=107./deg2rad, b_a=-8./deg2rad)
print v_LocalGroup(v_h, l, b, v_a=308., l_a=105./deg2rad, b_a=-7./deg2rad)
print v_h - 53. * np.cos(l) * np.cos(b) + 272. * np.sin(l) * np.cos(b) - 28. * np.sin(b)

3397.42249153
3468.02027936
3679.39508361
3434.09630082
3464.26096347


In [16]:
c1 = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')
c2 = SkyCoord(10.627, 41.2, frame='icrs', unit='deg')
sep = c2.separation(c1)
print c1
print c2

print sep.deg

<SkyCoord (ICRS): (ra, dec) in deg
    ( 10.625,  41.2)>
<SkyCoord (ICRS): (ra, dec) in deg
    ( 10.627,  41.2)>
0.00150482981776


In [68]:
v_t = 308.
l_t = 105./deg2rad
b_t = -7./deg2rad

print v_t * np.sin(b_t)

267.686332034
